<a href="https://colab.research.google.com/github/fistki/reinforcement/blob/main/SARSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow
# !pip install gym
# !pip install numpy

In [ ]:
import numpy as np
import pandas as pd
import random
from collections import defaultdict
import gym

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Agent:
    def __init__(self, num_states, num_actions):

    # update function

    # decrease_epsilon function

    # act function


In [ ]:
import gym
from gym import wrappers
env = gym.make('CartPole-v1', render_mode='rgb_array')
observation = env.reset()
agent = Agent(4,2)

In [ ]:
# training SARSA